In [1]:
import sys

sys.path.append("..")

from utils.ssh import exec, connect, exec_in_stream, exec_stream, save_token
from utils.text import print_output, print_title

client = connect('level0', 'level0')

Connect to level0
Execute command: sshpass -p level0 ssh level0@192.168.31.115 -p 4242 -oStrictHostKeyChecking=no


In [2]:
files_list = exec(client, 'ls', title='Get files list')
print_output(files_list, 'Files')
print_title('Test this file')

Get files list
Execute command: ls
Files: level0
Test this file


In [3]:
output = exec(client, './level0', err=True, title='Simple execute binary')
print_output(output, 'Output')
print_title('Nothing happened')

Simple execute binary
Execute command: ./level0
Nothing happened


In [4]:
output = exec(client, './level0 test', err=True, title='Simple execute binary')
print_output(output, 'Output')
print_title('Okay, expected another args, lets find it')
print_title('Debug Time')

Simple execute binary
Execute command: ./level0 test
Output: No !
Okay, expected another args, lets find it
Debug Time


In [5]:
main_structure = exec(client, 'echo "disass main" | gdb ./level0 -q', title='Get structure of main')
print_output(main_structure)
print_title('Many interesting things, cut a little')

Get structure of main
Execute command: echo "disass main" | gdb ./level0 -q
Reading symbols from /home/user/level0/level0...(no debugging symbols found)...done.
(gdb) Dump of assembler code for function main:
0x08048ec0 <+0>:	push   %ebp
0x08048ec1 <+1>:	mov    %esp,%ebp
0x08048ec3 <+3>:	and    $0xfffffff0,%esp
0x08048ec6 <+6>:	sub    $0x20,%esp
0x08048ec9 <+9>:	mov    0xc(%ebp),%eax
0x08048ecc <+12>:	add    $0x4,%eax
0x08048ecf <+15>:	mov    (%eax),%eax
0x08048ed1 <+17>:	mov    %eax,(%esp)
0x08048ed4 <+20>:	call   0x8049710 <atoi>
0x08048ed9 <+25>:	cmp    $0x1a7,%eax
0x08048ede <+30>:	jne    0x8048f58 <main+152>
0x08048ee0 <+32>:	movl   $0x80c5348,(%esp)
0x08048ee7 <+39>:	call   0x8050bf0 <strdup>
0x08048eec <+44>:	mov    %eax,0x10(%esp)
0x08048ef0 <+48>:	movl   $0x0,0x14(%esp)
0x08048ef8 <+56>:	call   0x8054680 <getegid>
0x08048efd <+61>:	mov    %eax,0x1c(%esp)
0x08048f01 <+65>:	call   0x8054670 <geteuid>
0x08048f06 <+70>:	mov    %eax,0x18(%esp)
0x08048f0a <+74>:	mov    0x1c(%esp),%e

In [6]:
for line in main_structure:
    if 'call' in line or 'cmp' in line:
        print_output(line)
print_title('Func get arg, call atoi to reformat it and call cmp to compare it with 0x1a7(423)')
print_title('Later execute command or write error')
print_title('Okay, we obtained the answer - 423 (compare), send it to our binary')

0x08048ed4 <+20>:	call   0x8049710 <atoi>
0x08048ed9 <+25>:	cmp    $0x1a7,%eax
0x08048ee7 <+39>:	call   0x8050bf0 <strdup>
0x08048ef8 <+56>:	call   0x8054680 <getegid>
0x08048f01 <+65>:	call   0x8054670 <geteuid>
0x08048f21 <+97>:	call   0x8054700 <setresgid>
0x08048f3d <+125>:	call   0x8054690 <setresuid>
0x08048f51 <+145>:	call   0x8054640 <execv>
0x08048f7b <+187>:	call   0x804a230 <fwrite>
Func get arg, call atoi to reformat it and call cmp to compare it with 0x1a7(423)
Later execute command and write file
Okay, we obtained the answer - 423 (compare), send it to our binary


In [7]:
stream = exec_stream('./level0 423', title='Execute binary with expected arg',
                     stdin=True, stdout=True, password='level0')
print_title('Okay, stdin intercepted, shell? Check it out')
output = exec_in_stream(stream, 'whoami', title='Try to call whoami')
print_output(output, 'Current user')
print_title('Level1, nice, find anything in home dir')

Execute binary with expected arg
Execute command: sshpass -p level0 ssh level0@192.168.31.115 -p 4242 -oStrictHostKeyChecking=no ./level0 423
Okay, stdin intercepted, shell? Check it out
Try to call whoami
Current user: level1
Level1, nice, find anything in home dir


In [8]:
stream = exec_stream('./level0 423', stdin=True, stdout=True, password='level0')
output = exec_in_stream(stream, 'ls -la /home/user/level1', title='Show home dir files')
print_output(output)
print_title('.pass file, got it')

Execute command: sshpass -p level0 ssh level0@192.168.31.115 -p 4242 -oStrictHostKeyChecking=no ./level0 423
Show home dir files
total 17
dr-xr-x---+ 1 level1 level1   80 Mar  6  2016 .
dr-x--x--x  1 root   root    340 Sep 23  2015 ..
-rw-r--r--  1 level1 level1  220 Apr  3  2012 .bash_logout
-rw-r--r--  1 level1 level1 3530 Sep 23  2015 .bashrc
-rwsr-s---+ 1 level2 users  5138 Mar  6  2016 level1
-rw-r--r--+ 1 level1 level1   65 Sep 23  2015 .pass
-rw-r--r--  1 level1 level1  675 Apr  3  2012 .profile
.pass file, it was written in main fun at step 187, read this


In [9]:
stream = exec_stream('./level0 423', stdin=True, stdout=True, password='level0')
token = exec_in_stream(stream, 'cat /home/user/level1/.pass', title='Read .pass file')
print_output(token, 'File content')
print_title('Woo-hoo!')

Execute command: sshpass -p level0 ssh level0@192.168.31.115 -p 4242 -oStrictHostKeyChecking=no ./level0 423
Read .pass file
File content: 1fe8a524fa4bec01ca4ea2a869af2a02260d4a7d5fe7e7c24d8617e6dca12d3a
Woo-hoo!


In [10]:
save_token(token)

Token to next level: 1fe8a524fa4bec01ca4ea2a869af2a02260d4a7d5fe7e7c24d8617e6dca12d3a


# client.close()